In [1]:
import torch
from gensim.models import KeyedVectors

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# to get the index of a token:
# wv.vocab[token].index

# in the paper hidden_size=50


class WordEncoder(torch.nn.Module):
    def __init__(self, embedding_weights, hidden_size):
        super(WordEncoder, self).__init__()
        self.embedding = torch.nn.Embedding.from_pretrained(embedding_weights, freeze=True)
        self.gru = torch.nn.GRU(
            input_size=self.embedding.embedding_dim, 
            hidden_size=hidden_size,
            bidirectional=True,
        )
    
    def forward(self, input, hidden):
        embedding = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedding, hidden)
        return output, hidden

In [18]:
wv = KeyedVectors.load("embedding/amazon_embedding.wv")
embedding_weights = torch.FloatTensor(wv.vectors)
word_encoder = WordEncoder(embedding_weights, hidden_size=50)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    word_encoder.parameters(), lr=0.001, momentum=0.9
)